## Meals Count Backend Utilities  
  
This notebook contains Python functionality to be incorporated into modules part of the [MealsCount](https://github.com/opensandiego/mealscount-backend) backend. **MealsCount** is an open-source project currently underway at [**OpenSanDiego**](https://opensandiego.org/), a [Code for America](https://www.codeforamerica.org/) brigade.  
  
**Note**: See [**[1]**](backend_utils_test.ipynb) for a detailed explanation of the core functionality, including intermediate outputs based on sample test data.   

In [2]:
import sys
import os
import pandas as pd
import numpy as np

from abc import ABC, abstractmethod

In [3]:
CWD = os.getcwd()

DATADIR = "data"
DATAFILE = "calpads_sample_data.xlsx"

In [4]:
class mcSchoolDistInput(ABC):
    """
    Base class for school district input. 
    """
    df = pd.DataFrame()
    
    def __init__(self):                
        pass
    
    @abstractmethod
    def to_frame(self):
        pass

In [5]:
def parseXL(self,xlfile):
    
    try:
        xl = pd.ExcelFile(xlfile)                
        tmpdf = xl.parse(xl.sheet_names[0])       
        
        #TODO
        
        self.df = tmpdf
        
    except Exception as e:
        raise e

In [6]:
class mcXLSchoolDistInput(mcSchoolDistInput):
    """
    Implementation for MealsCount Excel format school district input. Expects input 
    to be a file stored on the backend.
    """        
    
    def __init__(self, datafile):        
        mcSchoolDistInput.__init__(self)
        self.__datafile = datafile
        try:
            self.__parse(self.__datafile)
        except Exception as e:
            raise e
    
    def to_frame(self):
        return self.df
    
    __parse = parseXL           
        

In [7]:
data_in = mcXLSchoolDistInput(os.path.join(DATADIR,DATAFILE))
df = data_in.to_frame()
#df

### TODO  
  
* Add ParseXL functionality  
* Add API to get meta-data as a dictionary  